# Инициализация

Загружаем библиотеки необходимые для выполнения кода ноутбука.

In [1]:
import pandas as pd

# === ЭТАП 1 ===

# Загрузка первичных данных

Загружаем первичные данные из файлов:
- tracks.parquet
- catalog_names.parquet
- interactions.parquet

In [2]:


# Загрузка данных
tracks = pd.read_parquet('tracks.parquet')
catalog_names = pd.read_parquet('catalog_names.parquet')
interactions = pd.read_parquet('interactions.parquet')

# Обзор данных

Проверяем данные, есть ли с ними явные проблемы.

In [3]:
print(tracks.info())
print(tracks.isnull().sum())
print(tracks['track_id'].nunique() == len(tracks))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   track_id  1000000 non-null  int64 
 1   albums    1000000 non-null  object
 2   artists   1000000 non-null  object
 3   genres    1000000 non-null  object
dtypes: int64(1), object(3)
memory usage: 30.5+ MB
None
track_id    0
albums      0
artists     0
genres      0
dtype: int64
True


In [4]:
print(catalog_names.info())
print(catalog_names.isnull().sum())
print(catalog_names.groupby('type')['id'].nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812471 entries, 0 to 1812470
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   type    object
 2   name    object
dtypes: int64(1), object(2)
memory usage: 41.5+ MB
None
id      0
type    0
name    0
dtype: int64
type
album      658724
artist     153581
genre         166
track     1000000
Name: id, dtype: int64


In [5]:
print(interactions.info())
print(interactions.isnull().sum())
print(interactions.duplicated(subset=['user_id', 'track_id']).sum())

<class 'pandas.core.frame.DataFrame'>
Index: 222629898 entries, 0 to 291
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   user_id     int32         
 1   track_id    int32         
 2   track_seq   int16         
 3   started_at  datetime64[ns]
dtypes: datetime64[ns](1), int16(1), int32(2)
memory usage: 5.4 GB
None
user_id       0
track_id      0
track_seq     0
started_at    0
dtype: int64
0


# Выводы

Приведём выводы по первому знакомству с данными:
- есть ли с данными явные проблемы,
- какие корректирующие действия (в целом) были предприняты.

# === ЭТАП 2 ===

# EDA

Распределение количества прослушанных треков.

In [6]:
# Группируем по пользователям и считаем количество прослушиваний
user_plays = interactions.groupby('user_id').size().reset_index(name='tracks_played')
distribution = user_plays['tracks_played'].describe(percentiles=[.25, .5, .75, .9, .95, .99])
print("Распределение количества прослушанных треков:")
print(distribution)

Распределение количества прослушанных треков:
count    1.373221e+06
mean     1.621224e+02
std      3.512846e+02
min      1.000000e+00
25%      2.300000e+01
50%      5.500000e+01
75%      1.540000e+02
90%      3.890000e+02
95%      6.500000e+02
99%      1.576000e+03
max      1.663700e+04
Name: tracks_played, dtype: float64


Наиболее популярные треки

In [7]:
# Считаем количество прослушиваний для каждого трека
track_popularity = interactions.groupby('track_id').size().reset_index(name='plays')
top_tracks = track_popularity.sort_values('plays', ascending=False).head(10)
print("\nТоп-10 популярных треков:")
print(top_tracks)


Топ-10 популярных треков:
        track_id   plays
9098       53404  111062
483876  33311009  106921
26665     178529  101924
512157  35505245   99490
829320  65851540   86670
368072  24692821   86246
475289  32947997   85886
696106  51241318   85244
90461     795836   85042
647237  45499814   84748


Наиболее популярные жанры

In [9]:
# Объединяем прослушивания с треками для получения жанров
interactions_genres = interactions.merge(tracks[['track_id', 'genres']], on='track_id')

Треки, которые никто не прослушал

In [ ]:
# Находим треки из каталога, отсутствующие в прослушиваниях
unplayed_tracks = tracks[~tracks['track_id'].isin(interactions['track_id'])][['track_id']]
print("\nТреки без прослушиваний:")
print(unplayed_tracks)

# Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

# Сохранение данных

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

# Очистка памяти

Здесь, может понадобится очистка памяти для высвобождения ресурсов для выполнения кода ниже. 

Приведите соответствующие код, комментарии, например:
- код для удаление более ненужных переменных,
- комментарий, что следует перезапустить kernel, выполнить такие-то начальные секции и продолжить с этапа 3.

# === ЭТАП 3 ===

# Загрузка данных

Если необходимо, то загружаем items.parquet, events.parquet.

# Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.

# Топ популярных

Рассчитаем рекомендации как топ популярных.

# Персональные

Рассчитаем персональные рекомендации.

# Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций.

# Построение признаков

Построим три признака, можно больше, для ранжирующей модели.

# Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

# Оценка качества

Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

# === Выводы, метрики ===

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики.